In [80]:
%matplotlib inline
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [81]:
batch_size = 32
num_steps = 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

## one-hot编码

In [82]:
F.one_hot(torch.tensor([0, 2]), len(vocab))

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]])

## 小批量的数据形状是(批量大小， 时间步数)

In [83]:
X = torch.arange(10).reshape((2, 5))
print(X)
F.one_hot(X.T, 28).shape


tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])


torch.Size([5, 2, 28])

# 初始化循环神经网络模型的模型参数

In [84]:
def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size

    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01

    W_xh = normal((num_inputs, num_hiddens))
    W_hh = normal((num_hiddens, num_hiddens))
    b_h = torch.zeros(num_hiddens, device=device)
    W_hq = normal((num_hiddens, num_outputs)) # 隐变量到输出的
    b_q = torch.zeros(num_outputs, device=device)
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

In [85]:
# init_rnn_state函数在初始化时返回隐藏状态

In [86]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros(batch_size, num_hiddens, device=device),)

In [87]:
# 定义如何在一个时间步内计算隐藏状态和输出
def rnn(inputs, state, params):
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h) # H是前一个时刻的隐藏状态
        Y = torch.mm(H, W_hq) + b_q # 当前时刻预测下一时刻的输出
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H,)

In [88]:
# 创建一个类包装函数
class RNNModelScratch:
    """从零实现循环神经网络RNN"""
    def __init__(self, vocab_size, num_hiddens, device, get_params, init_state, forward_fn):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn

    def __call__(self, X, state): # __call__调用，可以看作是forward
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        return self.forward_fn(X, state, self.params)

    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

In [89]:
# 检查输出是否具有正确的形状
num_hiddens = 512
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params, init_rnn_state, rnn)
state = net.begin_state(X.shape[0], d2l.try_gpu())
Y, new_state = net(X.to(d2l.try_gpu()), state)
Y.shape, len(new_state), new_state[0].shape

(torch.Size([10, 28]), 1, torch.Size([2, 512]))

输出形状是（时间步数
批量大小，词表大小）， 而隐状态形状保持不变，即（批量大小，隐藏单元数）

In [90]:
# 定义一个预测函数来生成prefix之后的新字符

In [91]:
def predict_ch8(prefix, num_preds, net, vocab, device):
    """在prefix后生成新字符"""
    # num_preds 需要预测的数量
    state = net.begin_state(batch_size=1, device=device)
    outputs = [vocab[prefix[0]]] # output 是词在vocabulary对应的下标
    get_input = lambda :torch.tensor([outputs[-1]], device=device).reshape((1, 1)) # 把上一时刻的output当做当前时刻的输入 input
    for y in prefix[1:]:
        _, state = net(get_input(), state) # 此时有标准答案，因此不关心输出， 只需要更新状态，即H（隐变量）， 将隐藏状态预热
        outputs.append(vocab[y])
    for _ in range(num_preds):
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1))) # 取出最大值，因为是分类问题， one-hot编码
    return ''.join([vocab.idx_to_token[i] for i in outputs])

In [93]:
predict_ch8('time traveller', 10, net, vocab, d2l.try_gpu())

'time travellerpyvbvbvbvb'

# 梯度裁剪
$$\mathbf{g} \leftarrow \min\left(1, \frac{\theta}{\|\mathbf{g}\|}\right) \mathbf{g}.$$

In [94]:
def grad_clipping(net, theta):
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum(p.grad**2) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

# 定义一个函数在一个迭代周期内训练模型

In [ ]:
def train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter):
    """训练模型一个迭代周期"""
    state, timer = None, d2l.Timer()
    metric = d2l.Accumulator(2)
    for X, Y in train_iter:
        if state is None or use_random_iter:
            state = net.begin_state(batch_size=X.shape[0], device=device)
        else:
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                state.detach_()
            else:
                for s in state:
                    s.detach_()
